# 基本逻辑

1. 复制原理的内在逻辑：投资的**期望收益相同时，其投资成本也是相同**

2. 构造一个投资组合，股价上升时，投资组合的到期收益与期权的到期收入完全相同；股价下降时，投资组合的到期收益与期权的到期收入也完全相同，那么投资组合的成本就是期权的价格。


4. 如何构造这个投资组合，最简单的形式就是构建一个二元未知数方程，即$y=a*x-b*（1+r）$。其中x就是股价，是一个自变量，b是一个常量，很明显b更符合借款本金的形式，r是期限内无风险利率

5. 假定借入本金为b，购入股票数量为a股，初始时，购入股票所支付的资金等于自有资金+借入的借款本金。期限内无风险利率为r。其中自有资金就是投资组合成本，也就是期权的价格，也是我们最终求的结果。

6. 在股价上升和下降两种情形下，分别构建方程，分别求a和b，再用a*股价-b从而得到自有资金金额，即得到看涨期权的价格

> 复制组合由underlying和无风险资产组成，以上面为例：如果是多头underlying，则需要借入资金购入此资产。到期收益为underlying端减去借入的本金+利息

[原文链接](https://zhuanlan.zhihu.com/p/354474306)

# 模拟过程
假设ABC公司的股票现在的市价为50元。有1股以该股票为标的资产的看涨期权，执行价格为52.08元，到期时间是6个月。蒙卡模拟10000次，拟建立一个投资组合，包括购进适量的股票以及借入必要的款项，使得该组合6个月后的价值与购进该看涨期权的价值相等。

假定购进股票数量为a，借入本金为b

## 股价上升
$$a * S_T - b * (1 + r_f) = S_T - K$$

## 股价下跌
$$a * S_T - b * (1 + r_f) = 0$$

In [ ]:
import numpy as np
import sympy as sp
from exotic_pricing_HX import ExoticPricing
from py_vollib_vectorized import vectorized_black_scholes
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
S0 = 50
K = 52.08
T = 0.5
r = 0.05
sigma = 0.3
npath = 91


In [ ]:
MC = ExoticPricing(S0=S0,
                             K=K,
                             T=T,
                             r=r,
                             sigma=sigma,
                             simulation_rounds=int(10000),
                             npath=npath,
                             # fix_random_seed=True,
                             fix_random_seed=500)

In [ ]:
MC.stock_price_simulation()
ST = MC.terminal_prices
ST

In [ ]:
ST1 = (ST[ST > K]).mean()
ST2 = (ST[ST <= K]).mean()

In [ ]:
a = sp.Symbol('a')
b = sp.Symbol('b')
result = sp.solve([a * ST1 - b * (1 + r) - (ST1 - K), a * ST2 - b * (1 + r)],[a,b])
print(f'需要借入{result[b]}单位现金，并买入{result[a]}份标的')

In [ ]:
portfolio_cost = float(result[a] * S0 - result[b])
portfolio_cost

In [ ]:
vectorized_black_scholes('c',S0,K,T,r,sigma)